# BigQuery Exploration

In [ ]:
library(bigrquery)
library(repr)
bigrquery::bq_auth(path = "~/key-file")
project <- 'isb-cgc-outreach'

## SQL queries
SQL queries are fundamentally very similar to subsetting R data frames, where the syntax of subsetting an R data frame is as follows:

> *dataframe*[ ,*column* ][ *conditional* ]

The equivalent operation in SQL would look like this:

> SELECT *column* FROM *table* WHERE *conditional*



## Clinical and cohort queries
In this section we want to showcase how researchers can find clinical information of GDC cases. How can you quickly construct a cohort. As an example, how old are the patients for each case in a given project?

Tables used: *isb-cgc-bq.TCGA.clinical_gdc_current*

In [ ]:
# Our first query is to retrieve the columns present
column_sql <- "SELECT 
    column_name 
FROM isb-cgc-bq.TCGA.INFORMATION_SCHEMA.COLUMNS 
WHERE table_name = 'clinical_gdc_current'"

column_query <- bq_project_query(project, query=column_sql)
case_df <- as.vector(unlist(bq_table_download(column_query)))
paste(case_df, collapse=', ')

In [ ]:
# our second query retrieves the ages of all TCGA-BRCA cases
case_sql <- "SELECT
  submitter_id,
  demo__age_at_index,
  demo__vital_status,
FROM `isb-cgc-bq.TCGA.clinical_gdc_current`
WHERE proj__project_id = 'TCGA-BRCA'
"
case_query <- bq_project_query(project, query=case_sql)
case_df <- as.data.frame(bq_table_download(case_query))
head(case_df)

# Try substituting alternate or adding columns such as:
#   demo__vital_status
#   demo__race
# Find all the unique values that are in a column by querying only for:
#   distinct demo__race
# Try subseting the results by use of the where clause:
#   proj__project_id = 'TCGA-BRCA'
#   demo__age_at_index <= 70 AND demo__age_at_index >= 40

In [ ]:
# since we have the data in R we can directly generate visuals
options(repr.plot.width=7, repr.plot.height=5.5)
case_num <- length(unique(case_df[,2]))
hist(case_df[,2], breaks=case_num/2, main='age at index', xlab='age (years)')

In [ ]:
# including separating by vital status
hist(case_df[case_df[,3] == 'Alive',2], 
     breaks=case_num/2, xlim=c(25,90), 
     col='steelblue1', main='', xlab='age (years)')
hist(case_df[case_df[,3] == 'Dead',2], 
     breaks=case_num/2, add=1, col='orange')
legend('topright', bty='n', legend=c('Alive', 'Dead'), fill=c('steelblue1', 'orange'))

## Gene expression
Another common question researchers encounter is, given a gene of interest what is the expression pattern in cancers? Using the GDC expression tables it's possible to retrieve huge amounts of information and do high level comparisons quickly.

Tables used: *isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current*

In [ ]:
# a query to retrieve FPKM_UQ values for one specific gene (CCNB1)
expr_sql <- "SELECT
  case_barcode,
  aliquot_barcode,
  sample_type_name,
  HTSeq__FPKM_UQ
FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
WHERE project_short_name = 'TCGA-BRCA'
  AND gene_name = 'BRCA1' #'CCNB1'"

expr_query <- bq_project_query(project, query=expr_sql)
expr_df <- bq_table_download(expr_query)
head(expr_df)

# try changing the project_short_name to 'TCGA-LUAD'
# or substitute HTSeq__FPKM_UQ to retrieve other measures
#    HTSeq__counts
#    HTSeq__FPKM
# change the gene name
#    BRCA1
#    BRCA2
#    FOXA1

In [ ]:
options(repr.plot.width=5, repr.plot.height=6)
tumor <- unlist(expr_df[expr_df[,3] == 'Primary Tumor',4])
normal <- unlist(expr_df[expr_df[,3] == 'Solid Tissue Normal',4])
metastatic <- unlist(expr_df[expr_df[,3] == 'Metastatic',4])

boxplot(list(normal, tumor, metastatic), names=c('normal', 'tumor', 'metastatic'), main='CCNB1 expression', ylab='upper-quartile fpkm')

As the size of the data of interest increases, calculating statistics on a local machine may become prohibitive. One of the benefits of BigQuery is that it scales compute power to the demands of the query. We can functions built into BQ to generate summary statistics.

In [ ]:
# This query will calculate averages, standard deviations, maximums, and minimums
summ_sql <- "SELECT
  project_short_name,
  sample_type_name,
  count( distinct aliquot_barcode ) count_aliq,
  avg(HTSeq__FPKM_UQ) avg_fpkm,
  stddev(HTSeq__FPKM_UQ) sdv_fpkm,
  max(HTSeq__FPKM_UQ) max_fpkm,
  min(HTSeq__FPKM_UQ) min_fpkm
FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
WHERE gene_name = 'CCNB1'
GROUP BY project_short_name, sample_type_name
"

summ_query <- bq_project_query(project, query=summ_sql)
summ_df <- bq_table_download(summ_query)
head(summ_df)

# Other functions such as count and distinct can add useful information
#    count( distinct aliquot_barcode )
# Retrieve results for all genes or for all projects by
# removing gene_name from the WHERE clause 
# and adding it to SELECT and GROUP BY

In [ ]:
# summ_sql <- "SELECT
#  sample_type_name,
#  project_short_name,
#  avg(HTSeq__FPKM_UQ) avg_fpkm,
#  stddev(HTSeq__FPKM_UQ) sdv_fpkm,
#  max(HTSeq__FPKM_UQ) max_fpkm,
#  min(HTSeq__FPKM_UQ) min_fpkm
#FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
#WHERE gene_name = 'CCNB1'
#GROUP BY project_short_name,sample_type_name"

## Genome annotation
If we need annotatio information about a gene it's possible to quickly retrieve Gencode annotation from ISB-CGC hosted BigQuery tables.

Table: *isb-cgc-bq.GENCODE_versioned.annotation_gtf_hg38_v22*

In [ ]:
annot_sql <- "SELECT * 
FROM `isb-cgc-bq.GENCODE_versioned.annotation_gtf_hg38_v22`
WHERE gene_name = 'CCNB1' 
  AND transcript_name = 'CCNB1-001'"

annot_query <- bq_project_query(project, query=annot_sql)
annot_df <- bq_table_download(annot_query)
annot_df

## Mutation data
You may encounter a mutation and wonder if it's represented in other cancer data. Given the gene annotation data from above we can query the somatic mutation table for mutations within a gene.

Table: *isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current*

In [ ]:
# this query retrieves all mutations falling between the start codon and the stop codon of our gene.
mut_sql <- "SELECT case_barcode, 
    Chromosome, 
    Start_Position, 
    End_Position, 
    Reference_Allele, 
    Tumor_Seq_Allele1, 
    Tumor_Seq_Allele2
FROM `isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current`
WHERE Chromosome = 'chr5'
    AND Start_Position >= 69167263
    AND End_Position <= 69177631"

mut_query <- bq_project_query(project, query=mut_sql)
mut_df <- bq_table_download(mut_query)
head(mut_df)
nrow(mut_df)

In [ ]:
# we can plot the positions and number of mutations on the chromosome
options(repr.plot.width=15, repr.plot.height=4)
mut_count <- table(mut_df[,3])
max_count <- max(mut_count)
plot(NA, xlim=c(min(mut_df[,3])-1000, max(mut_df[,3])+1000), ylim=c(-0.3,max_count), 
     yaxt='n', bty='n', main='', xlab='Chr5 position', ylab='occurance count')
jnk <- apply(annot_df[annot_df$feature == 'exon',], 1, function(row){
    rect(row[4], -0.2, row[5], -0.1, col='green')
})
axis(2, at=seq(0,3,1))
jnk <- sapply(1:length(mut_count), function(ix){
    x <- as.numeric(names(mut_count)[ix])
    segments(x, 0, x, mut_count[ix])
    })

In [ ]:
sql <- "SELECT
    case_barcode, 
    Chromosome, 
    Start_Position, 
    End_Position, 
    Reference_Allele, 
    Tumor_Seq_Allele1, 
    Tumor_Seq_Allele2
FROM `isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current`
WHERE 
    Chromosome = 'chr5'
    AND ( (Start_Position < 69067135 and Start_Position > 69167135)
        or (End_Position > 69177358 and End_Position < 69277358) )"

query <- bq_project_query(project, query=sql)
df <- bq_table_download(query)
head(df)

## Table joins
Many use cases for researchers require combined data from multiple of the tables we worked with in the previous examples. How do we join these data using BigQuery?

Tables:
*   *isb-cgc-bq.TCGA.clinical_gdc_current*
*   *isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current*

In [ ]:
sql <- "SELECT
    mut.case_barcode,
    mut.Chromosome, 
    mut.Start_Position, 
    mut.End_Position, 
    mut.Reference_Allele, 
    mut.Tumor_Seq_Allele1, 
    mut.Tumor_Seq_Allele2, 
    clin.demo__vital_status 
FROM `isb-cgc-bq.TCGA.clinical_gdc_current` clin
JOIN `isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current` mut 
    ON clin.submitter_id = mut.case_barcode
WHERE mut.project_short_name = 'TCGA-BRCA'"

query <- bq_project_query(project, query=sql)
df <- bq_table_download(query)
head(df)

In [ ]:
options(repr.plot.width=5, repr.plot.height=6)
muts_alive <- as.vector(table(df[df['demo__vital_status'] == 'Alive','case_barcode']))
muts_dead <- as.vector(table(df[df['demo__vital_status'] == 'Dead','case_barcode']))
boxplot(muts_alive, muts_dead, ylim=c(0,1000), 
  names=c('Alive', 'Dead'), ylab='mutations')